In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# ── 1) Hyperparameters ─────────────────────────────────────────────────────────
TRAIN_LEN   = 400    # train on steps [0..399]
BATCH_SIZE  = 10
HIDDEN_SIZE = 64
NUM_LAYERS  = 2
DROPOUT     = 0.2
LR          = 1e-3
NUM_EPOCHS  = 15
RANDOM_SEED = 42

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ── 2) Load, clean & shift labels ───────────────────────────────────────────────
df = pd.read_csv("features_all_models_FINAL.csv")
#  a) drop first 100 null‐warmups
df = df.groupby("inst", group_keys=False) \
       .apply(lambda g: g.iloc[100:]) \
       .reset_index(drop=True)
#  b) shift labels *one day ahead* per instrument
df["true_regime"] = df.groupby("inst")["true_regime"].shift(-1)
#  c) drop the last row of each instrument (now NaN label)
df = df.dropna(subset=["true_regime"]).reset_index(drop=True)
df["true_regime"] = df["true_regime"].astype(int)

price_df = pd.read_csv("prices.txt", sep=r"\s+", header=None)

# ── 3) Determine sequence length & build arrays ────────────────────────────────
seq_lens = df.groupby("inst").size()
SEQ_LEN  = int(seq_lens.max())
print("Detected sequence length per instrument (post-shift):", SEQ_LEN)

n_inst    = df["inst"].nunique()
feat_cols = [c for c in df.columns if c not in ("inst","time","true_regime")]

# initialize
X = np.zeros((n_inst, SEQ_LEN, len(feat_cols)), dtype=np.float32)
Y = np.zeros((n_inst, SEQ_LEN),               dtype=np.int64)

# fill per-instrument
for inst in range(n_inst):
    sub = df[df["inst"]==inst].reset_index(drop=True)
    assert len(sub)==SEQ_LEN
    X[inst] = sub[feat_cols].values
    Y[inst] = sub["true_regime"].values

NUM_TAGS = int(Y.max()) + 1

# ── 4) Split into train vs. test windows ──────────────────────────────────────
X_train = torch.tensor(X[:, :TRAIN_LEN, :])
Y_train = torch.tensor(Y[:, :TRAIN_LEN])
X_test  = torch.tensor(X[:, TRAIN_LEN:, :])
Y_test  = Y[:, TRAIN_LEN:]           # numpy for metrics & plotting
LEN_TEST = SEQ_LEN - TRAIN_LEN

# ── 5) Dataset & DataLoader ───────────────────────────────────────────────────
class SeqTagDataset(Dataset):
    def __init__(self, X, y):
        self.X = X; self.y = y
    def __len__(self):
        return self.X.size(0)
    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_loader = DataLoader(SeqTagDataset(X_train, Y_train),
                          batch_size=BATCH_SIZE, shuffle=True)

# ── 6) BiLSTM tagger ───────────────────────────────────────────────────────────
class BiLSTMTagger(nn.Module):
    def __init__(self, feat_dim, hidden_dim, num_layers, num_tags, dropout):
        super().__init__()
        self.lstm = nn.LSTM(feat_dim, hidden_dim,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True,
                            dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, num_tags)
    def forward(self, x):
        out, _ = self.lstm(x)     # (B, T, 2H)
        return self.fc(out)       # (B, T, num_tags)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = BiLSTMTagger(
    feat_dim   = X.shape[2],
    hidden_dim = HIDDEN_SIZE,
    num_layers = NUM_LAYERS,
    num_tags   = NUM_TAGS,
    dropout    = DROPOUT
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# ── 7) Train ───────────────────────────────────────────────────────────────────
for epoch in range(1, NUM_EPOCHS+1):
    model.train()
    total_loss = 0.0
    for feats, tags in train_loader:
        feats, tags = feats.to(device), tags.to(device)
        logits      = model(feats)              # (B, T, C)
        loss        = criterion(
            logits.view(-1, NUM_TAGS),         # (B*T, C)
            tags.view(-1)                      # (B*T,)
        )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch:02d} — Avg Loss: {total_loss/len(train_loader):.4f}")

# ── 8) Inference + metrics + two‐panel plotting ───────────────────────────────
def get_segments(reg):
    changes = np.flatnonzero(reg[1:] != reg[:-1])
    starts  = np.concatenate(([0], changes+1))
    ends    = np.concatenate((changes, [len(reg)-1]))
    return list(zip(starts, ends, reg[starts]))

true_cmap = ListedColormap(["#ff0000","#808080","#00ff00"])
pred_cmap = ListedColormap(["#cc0000","#444444","#00cc00"])

model.eval()
with torch.no_grad():
    logits_test = model(X_test.to(device))      # (50, LEN_TEST, C)
    preds_test  = logits_test.argmax(dim=2).cpu().numpy()

for inst in range(n_inst):
    true_seq = Y_test[inst]
    pred_seq = preds_test[inst]
    price    = price_df.iloc[100+TRAIN_LEN:100+TRAIN_LEN+LEN_TEST, inst].values

    acc = (pred_seq == true_seq).mean()
    print(f"Inst {inst:02d} Test acc: {acc:.3f}")

    fig, (ax1, ax2) = plt.subplots(2,1, sharex=True, figsize=(12,6))

    # TRUE regimes
    for s,e,lbl in get_segments(true_seq):
        ax1.axvspan(s, e, color=true_cmap(lbl), alpha=0.5, linewidth=0)
    ax1.plot(price, 'k-', label='Price')
    ax1.set_title(f"Inst {inst} — TRUE regimes (t={TRAIN_LEN}→end)")
    ax1.legend(loc='upper right')

    # PREDICTED regimes
    for s,e,lbl in get_segments(pred_seq):
        ax2.axvspan(s, e, color=pred_cmap(lbl), alpha=0.5, linewidth=0)
    ax2.plot(price, 'k-', label='Price')
    ax2.set_title(f"Inst {inst} — PREDICTED regimes (t={TRAIN_LEN}→end)")
    ax2.legend(loc='upper right')

    plt.tight_layout()
    plt.show()


In [ ]:
# regime_inference.py  ——  ONLINE VERSION (Option B aligned)
# ==========================================================
import numpy as np
import pandas as pd

# ───────────────────────── helpers ───────────────────────────────────────
def _ols_slope(y: np.ndarray) -> float:
    """OLS slope of the vector y against a time index 0…len(y)-1."""
    t = np.arange(len(y))
    X = np.vstack([t, np.ones_like(t)]).T
    m, _ = np.linalg.lstsq(X, y, rcond=None)[0]
    return m


# ─── Slope/Vol regime  (★ causal 100-bar median, Option B) ───────────────
def _slope_vol_reg(close: np.ndarray,
                   idx:   int,
                   slope_win: int = 30,
                   vol_win:   int = 100) -> float | int:
    """
    Classifies the bar *idx* in `close`:
        2 = bull  (slope>0 and 1-day σ below rolling 100-bar median)
        0 = bear
        np.nan if any field NaN
    The median rule is IDENTICAL to build_feature_matrix (Option B).
    """
    logp = np.log(close)

    # rolling slope
    slope_series = (
        pd.Series(logp)
          .rolling(slope_win, min_periods=slope_win)
          .apply(lambda arr: _ols_slope(arr), raw=True)
    )

    # rolling volatility (σ of log-returns over `vol_win`)
    rtn         = pd.Series(logp).diff()
    vol_series  = rtn.rolling(vol_win, min_periods=vol_win).std()

    slope = slope_series.iloc[idx]
    vol   = vol_series.iloc[idx]
    if np.isnan(slope) or np.isnan(vol):
        return np.nan

    # causal 100-bar rolling median *of vol*
    median_vol = (
        vol_series
          .rolling(window=100, min_periods=100)
          .median()
          .iloc[idx]
    )

    return 2 if (slope > 0 and vol < median_vol) else 0


# ────────────────────── pipeline (no drop_last) ──────────────────────────
def compute_regime_features_window(prices_window: np.ndarray) -> np.ndarray:
    """
    Input
    -----
    prices_window : np.ndarray, shape (n_inst, win_len)
        Last `win_len` closes for each instrument (win_len ≥ 100).

    Output
    ------
    np.ndarray, shape (n_inst, 9)
        Columns ordered: [ma, ema, slope_vol, macd, kalman,
                          fib, psar, zscore, wret]
    """
    n_inst, win_len = prices_window.shape
    idx = win_len - 1                     # evaluate at the newest bar

    out = np.full((n_inst, 9), np.nan)
    sqrt_w = np.arange(1, 46, dtype=float) ** 0.5
    sqrt_w /= sqrt_w.sum()

    for i in range(n_inst):
        close = prices_window[i]
        logp  = np.log(close)

        # MA regime
        ma_s  = pd.Series(logp).rolling(5).mean().iloc[idx]
        ma_l  = pd.Series(logp).rolling(70).mean().iloc[idx]
        ma_reg = 0 if ma_l > ma_s else 2

        # EMA regime
        ema_s = pd.Series(logp).ewm(span=5,  adjust=False).mean().iloc[idx]
        ema_l = pd.Series(logp).ewm(span=50, adjust=False).mean().iloc[idx]
        ema_reg = 2 if ema_s > ema_l else 0

        # Slope/Vol regime
        sv_reg = _slope_vol_reg(close, idx)

        # MACD regime
        macd_line = (
            pd.Series(logp).ewm(50, adjust=False).mean()
          - pd.Series(logp).ewm(90, adjust=False).mean()
        )
        signal_line = macd_line.ewm(span=40, adjust=False).mean()
        macd_reg = 2 if macd_line.iloc[idx] > signal_line.iloc[idx] else 0

        # Kalman trend regime
        proc_var, meas_var = 0.01, 10.0
        x_est = np.zeros(win_len)
        P     = np.zeros(win_len)
        x_est[0], P[0] = logp[0], 1.0
        for t in range(1, win_len):
            x_pred = x_est[t-1]
            P_pred = P[t-1] + proc_var
            K      = P_pred / (P_pred + meas_var)
            x_est[t] = x_pred + K * (logp[t] - x_pred)
            P[t]     = (1 - K) * P_pred
        kalman_reg = 2 if logp[idx] > x_est[idx] else 0

        # Fibonacci regime
        if idx >= 50:
            win50 = close[idx-49 : idx+1]
            hi, lo = win50.max(), win50.min()
            rng = hi - lo
            upper, lower = lo + 0.786*rng, lo + 0.618*rng
            fib_reg = 2 if close[idx] > upper else 0 if close[idx] < lower else 1
        else:
            fib_reg = np.nan

        # PSAR regime
        psar = np.empty(win_len)
        trend_up, af, max_af = True, 0.01, 0.10
        ep = close[0]
        psar[0] = close[0]
        for t in range(1, win_len):
            psar[t] = psar[t-1] + af*(ep - psar[t-1])
            if trend_up:
                if close[t] < psar[t]:
                    trend_up, psar[t], ep, af = False, ep, close[t], 0.01
                elif close[t] > ep:
                    ep, af = close[t], min(af+0.01, max_af)
            else:
                if close[t] > psar[t]:
                    trend_up, psar[t], ep, af = True, ep, close[t], 0.01
                elif close[t] < ep:
                    ep, af = close[t], min(af+0.01, max_af)
        psar_reg = 2 if close[idx] > psar[idx] else 0

        # Z-score regime
        ma90 = pd.Series(close).rolling(90).mean().iloc[idx]
        sd90 = pd.Series(close).rolling(90).std().iloc[idx]
        if np.isnan(ma90) or np.isnan(sd90):
            zscore_reg = np.nan
        else:
            z = (close[idx] - ma90) / sd90
            zscore_reg = 2 if z > 0.5 else 0 if z < -0.5 else 1

        # Weighted-return regime
        if idx >= 45:
            r  = pd.Series(close).pct_change().iloc[idx-44 : idx+1].values
            wr = np.dot(r, sqrt_w)
            wret_reg = 2 if wr > 0 else 0 if wr < 0 else 1
        else:
            wret_reg = np.nan

        out[i] = [
            ma_reg, ema_reg, sv_reg, macd_reg, kalman_reg,
            fib_reg, psar_reg, zscore_reg, wret_reg
        ]

    return out


# ──────────────────── I/O wrappers for prices.txt ───────────────────────
def _extract_window(price_file: str,
                    timestep: int,
                    win_len: int = 100) -> np.ndarray:
    """
    Extract the last `win_len` closes (inclusive) ending at `timestep`
    and return an array with shape (n_inst, win_len).
    """
    df = pd.read_csv(price_file, sep=r"\s+", header=None)
    n_rows, _ = df.shape

    if not (0 <= timestep < n_rows):
        raise ValueError(f"timestep {timestep} out of range (0 … {n_rows-1})")
    if timestep < win_len - 1:
        raise ValueError("Not enough history to build a 100-bar window.")

    slice_df = df.iloc[timestep-win_len+1 : timestep+1, :]
    return slice_df.to_numpy().T      # (n_inst, win_len)


def infer_from_file(price_file: str,
                    timestep:   int) -> np.ndarray:
    """
    Convenience:
      • read prices.txt
      • build a (n_inst, 100) window ending at `timestep`
      • feed through compute_regime_features_window
    """
    window = _extract_window(price_file, timestep, win_len=100)  # ← 100, not 102
    return compute_regime_features_window(window)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.linear_model import LogisticRegression

# ─────────────────────────────────────────────────────────────────────
# 0)  Prepare training data  (X_train, Y_train already in your session)
# ─────────────────────────────────────────────────────────────────────
X_flat = X_train.cpu().numpy().reshape(-1, X_train.shape[2])   # (N, 9)
y_flat = Y_train.cpu().numpy().reshape(-1)                     # (N,)

# Impute NaNs with column means (fits Option-B causal warm-up behaviour)
col_means   = np.nanmean(X_flat, axis=0)
X_flat_imp  = np.where(np.isnan(X_flat), col_means, X_flat)

# ─────────────────────────────────────────────────────────────────────
# 1)  Fit proxy model
# ─────────────────────────────────────────────────────────────────────
lr = LogisticRegression(
    penalty=None,
    solver='saga',
    max_iter=20_000,
    class_weight='balanced',
    multi_class='ovr'
)
lr.fit(X_flat_imp, y_flat)
print("✅ Logistic proxy trained on LSTM features")

# ─────────────────────────────────────────────────────────────────────
# 2)  Streaming inference
# ─────────────────────────────────────────────────────────────────────  # after your Option-B update

price_df = pd.read_csv("prices.txt", sep=r"\s+", header=None)
n_rows, n_inst = price_df.shape
WIN_LEN = 100          # ← must match infer_from_file()’s 100-bar window

preds_all = np.zeros((n_inst, n_rows), dtype=int)

for t in range(WIN_LEN-1, n_rows):
    if (t - (WIN_LEN-1)) % 50 == 0 or t in (WIN_LEN-1, n_rows-1):
        print(f"Inferring at window end t={t}  (bar {t+1}/{n_rows})")

    feats_t = infer_from_file("prices.txt", timestep=t)        # (n_inst, 9)
    feats_t = np.where(np.isnan(feats_t), col_means, feats_t)  # impute

    regs_t  = lr.predict(feats_t)
    preds_all[:, t] = regs_t

print("✅ Inference complete!")

# ─────────────────────────────────────────────────────────────────────
# 3)  Visualisation helper
# ─────────────────────────────────────────────────────────────────────
def _segments(labels):
    ch = np.flatnonzero(labels[:-1] != labels[1:])
    starts = np.concatenate(([0], ch + 1))
    ends   = np.concatenate((ch, [len(labels)-1]))
    return zip(starts, ends, labels[starts])

cmap = ListedColormap(["#ffcccc", "#ccffcc"])   # 0→bear, 2→bull

for inst in range(n_inst):
    price = price_df.iloc[:, inst].values
    reg   = preds_all[inst]

    fig, ax = plt.subplots(figsize=(12, 3))
    for s, e, lbl in _segments(reg):
        ax.axvspan(s, e, color=cmap(lbl // 2), alpha=0.35)
    ax.plot(price, 'k-', lw=1)
    ax.set_title(f"Instrument {inst:02d}")
    ax.set_xlim(WIN_LEN-1, n_rows-1)
    ax.set_xlabel("Timestep")
    ax.set_ylabel("Price")
    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.linear_model import LogisticRegression

# ─────────────────────────────────────────────────────────────────────
# 0)  (unchanged) Prepare training data  (X_train, Y_train in session)
# ─────────────────────────────────────────────────────────────────────
X_flat = X_train.cpu().numpy().reshape(-1, X_train.shape[2])
y_flat = Y_train.cpu().numpy().reshape(-1)

col_means  = np.nanmean(X_flat, axis=0)
X_flat_imp = np.where(np.isnan(X_flat), col_means, X_flat)

# ─────────────────────────────────────────────────────────────────────
# 1)  (unchanged) Fit proxy model
# ─────────────────────────────────────────────────────────────────────
lr = LogisticRegression(
    penalty=None,
    solver='saga',
    max_iter=20_000,
    class_weight='balanced',
    multi_class='ovr'
)
lr.fit(X_flat_imp, y_flat)
print("✅ Logistic proxy trained on LSTM features")

# ─────────────────────────────────────────────────────────────────────
# 2)  Streaming inference, but only print last LAST calls
# ─────────────────────────────────────────────────────────────────────
price_df = pd.read_csv("prices.txt", sep=r"\s+", header=None)
n_rows, n_inst = price_df.shape
WIN_LEN = 100
LAST    = 350

preds_all = np.zeros((n_inst, n_rows), dtype=int)

for t in range(WIN_LEN-1, n_rows):
    feats_t = infer_from_file("prices.txt", timestep=t)  
    feats_t = np.where(np.isnan(feats_t), col_means, feats_t)
    preds_all[:, t] = lr.predict(feats_t)

    # only print the last LAST calls:
    if t >= n_rows - LAST:
        print(f"Inferring at window end t={t}  (bar {t+1}/{n_rows})")

print("✅ Inference complete!")

# ─────────────────────────────────────────────────────────────────────
# 3)  Plot only the final LAST points
# ─────────────────────────────────────────────────────────────────────
def _segments(labels):
    ch     = np.flatnonzero(labels[:-1] != labels[1:])
    starts = np.concatenate(([0], ch + 1))
    ends   = np.concatenate((ch, [len(labels)-1]))
    return zip(starts, ends, labels[starts])

cmap = ListedColormap(["#ffcccc", "#ccffcc"])   # 0→bear, 2→bull

start = n_rows - LAST
x_vals = np.arange(start, n_rows)  # for proper x-axis

for inst in range(n_inst):
    price_seg = price_df.iloc[start:, inst].values
    reg_seg   = preds_all[inst, start:]

    fig, ax = plt.subplots(figsize=(12, 3))
    for s, e, lbl in _segments(reg_seg):
        ax.axvspan(x_vals[s], x_vals[e], color=cmap(lbl // 2), alpha=0.35)
    ax.plot(x_vals, price_seg, 'k-', lw=1)
    ax.set_title(f"Instrument {inst:02d} (last {LAST} bars)")
    ax.set_xlim(start, n_rows-1)
    ax.set_xlabel("Timestep")
    ax.set_ylabel("Price")
    plt.tight_layout()
    plt.show()


In [ ]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# ── 1) Hyperparameters ─────────────────────────────────────────────────────────
TRAIN_LEN   = 400
BATCH_SIZE  = 10
HIDDEN_SIZE = 128
NUM_LAYERS  = 2
NUM_HEADS   = 4      # added for transformer
DROPOUT     = 0.2
LR          = 1e-3
NUM_EPOCHS  = 12
RANDOM_SEED = 42

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ── 2) Load, clean & shift labels ───────────────────────────────────────────────
df = pd.read_csv("features_all_models_FINAL.csv")
df = df.groupby("inst", group_keys=False).apply(lambda g: g.iloc[100:]).reset_index(drop=True)
df["true_regime"] = df.groupby("inst")["true_regime"].shift(-1)
df = df.dropna(subset=["true_regime"]).reset_index(drop=True)
df["true_regime"] = df["true_regime"].astype(int)
price_df = pd.read_csv("prices.txt", sep=r"\s+", header=None)

# ── 3) Determine sequence length & build arrays ────────────────────────────────
seq_lens = df.groupby("inst").size()
SEQ_LEN  = int(seq_lens.max())
print("Detected sequence length per instrument (post-shift):", SEQ_LEN)

n_inst    = df["inst"].nunique()
feat_cols = [c for c in df.columns if c not in ("inst","time","true_regime")]

X = np.zeros((n_inst, SEQ_LEN, len(feat_cols)), dtype=np.float32)
Y = np.zeros((n_inst, SEQ_LEN),               dtype=np.int64)

for inst in range(n_inst):
    sub = df[df["inst"]==inst].reset_index(drop=True)
    X[inst] = sub[feat_cols].values
    Y[inst] = sub["true_regime"].values

NUM_TAGS = int(Y.max()) + 1

# ── 4) Split into train vs. test windows ──────────────────────────────────────
X_train = torch.tensor(X[:, :TRAIN_LEN, :])
Y_train = torch.tensor(Y[:, :TRAIN_LEN])
X_test  = torch.tensor(X[:, TRAIN_LEN:, :])
Y_test  = Y[:, TRAIN_LEN:]
LEN_TEST = SEQ_LEN - TRAIN_LEN

# ── 5) Dataset & DataLoader ───────────────────────────────────────────────────
class SeqTagDataset(Dataset):
    def __init__(self, X, y):
        self.X = X; self.y = y
    def __len__(self):
        return self.X.size(0)
    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_loader = DataLoader(SeqTagDataset(X_train, Y_train),
                          batch_size=BATCH_SIZE, shuffle=True)

# ── Positional Encoding for Transformer ───────────────────────────────────────
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.2, max_len=1000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()
                             * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe.unsqueeze(1))  # (max_len, 1, d_model)

    def forward(self, x):
        # x: (seq_len, batch, d_model)
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)

# ── 6) Transformer ──────────────────────────────────────────────────────
class TransformerTagger(nn.Module):
    def __init__(self, feat_dim, hidden_dim, num_layers, num_heads, dropout, num_tags, max_seq_len):
        super().__init__()
        self.input_proj = nn.Linear(feat_dim, hidden_dim)
        self.pos_encoder = PositionalEncoding(hidden_dim, dropout, max_seq_len)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dropout=dropout,
            batch_first=False
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, num_tags)

    def forward(self, x):
        x = self.input_proj(x)                     
        x = x.permute(1, 0, 2)                      
        x = self.pos_encoder(x)                     
        out = self.transformer_encoder(x)           
        out = out.permute(1, 0, 2)                  
        return self.fc(out)                         

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerTagger(
    feat_dim   = X.shape[2],
    hidden_dim = HIDDEN_SIZE,
    num_layers = NUM_LAYERS,
    num_heads  = NUM_HEADS,
    dropout    = DROPOUT,
    num_tags   = NUM_TAGS,
    max_seq_len= SEQ_LEN
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# ── 7) Train ───────────────────────────────────────────────────────────────────
for epoch in range(1, NUM_EPOCHS+1):
    model.train()
    total_loss = 0.0
    for feats, tags in train_loader:
        feats, tags = feats.to(device), tags.to(device)
        logits      = model(feats)                   
        loss        = criterion(
            logits.view(-1, NUM_TAGS),             
            tags.view(-1)                          
        )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch:02d} — Avg Loss: {total_loss/len(train_loader):.4f}")

# ── 8) Inference + metrics + two‐panel plotting ───────────────────────────────
def get_segments(reg):
    changes = np.flatnonzero(reg[1:] != reg[:-1])
    starts  = np.concatenate(([0], changes+1))
    ends    = np.concatenate((changes, [len(reg)-1]))
    return list(zip(starts, ends, reg[starts]))

true_cmap = ListedColormap(["#ff0000","#808080","#00ff00"])
pred_cmap = ListedColormap(["#cc0000","#444444","#00cc00"])

model.eval()
with torch.no_grad():
    logits_test = model(X_test.to(device))       # (n_inst, LEN_TEST, C)
    preds_test  = logits_test.argmax(dim=2).cpu().numpy()

for inst in range(n_inst):
    true_seq = Y_test[inst]
    pred_seq = preds_test[inst]
    price    = price_df.iloc[100+TRAIN_LEN:100+TRAIN_LEN+LEN_TEST, inst].values

    acc = (pred_seq == true_seq).mean()
    print(f"Inst {inst:02d} Test acc: {acc:.3f}")

    fig, (ax1, ax2) = plt.subplots(2,1, sharex=True, figsize=(12,6))
    for s,e,lbl in get_segments(true_seq):
        ax1.axvspan(s, e, color=true_cmap(lbl), alpha=0.5, linewidth=0)
    ax1.plot(price, 'k-', label='Price')
    ax1.set_title(f"Inst {inst} — TRUE regimes (t={TRAIN_LEN}→end)")
    ax1.legend(loc='upper right')

    for s,e,lbl in get_segments(pred_seq):
        ax2.axvspan(s, e, color=pred_cmap(lbl), alpha=0.5, linewidth=0)
    ax2.plot(price, 'k-', label='Price')
    ax2.set_title(f"Inst {inst} — PREDICTED regimes (t={TRAIN_LEN}→end)")
    ax2.legend(loc='upper right')

    plt.tight_layout()
    plt.show()


In [ ]:
# Full Jupyter cell: k‐step‐ahead UniLSTM with no future leakage

%matplotlib inline
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# ── Hyperparameters ────────────────────────────────────────────────────────────
TRAIN_LEN   = 400
BATCH_SIZE  = 10
HIDDEN_SIZE = 64
NUM_LAYERS  = 2
DROPOUT     = 0.2
LR          = 1e-3
NUM_EPOCHS  = 15
RANDOM_SEED = 42
HORIZON     = 5    # predict regime 5 bars ahead

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ── Load & prepare data ────────────────────────────────────────────────────────
df = pd.read_csv("features_all_models_FINAL.csv")
# drop first 100 warmup
df = df.groupby("inst", group_keys=False).apply(lambda g: g.iloc[100:]).reset_index(drop=True)
# shift labels by HORIZON ahead
df["target_regime"] = df.groupby("inst")["true_regime"].shift(-HORIZON)
df = df.dropna(subset=["target_regime"]).reset_index(drop=True)
df["target_regime"] = df["target_regime"].astype(int)

price_df = pd.read_csv("prices.txt", sep=r"\s+", header=None)

# build sequences
seq_lens = df.groupby("inst").size()
SEQ_LEN  = int(seq_lens.max())
n_inst   = df["inst"].nunique()
feat_cols = [c for c in df.columns if c not in ("inst","time","true_regime","target_regime")]

X = np.zeros((n_inst, SEQ_LEN, len(feat_cols)), dtype=np.float32)
Y = np.zeros((n_inst, SEQ_LEN),               dtype=np.int64)

for inst in range(n_inst):
    sub = df[df["inst"]==inst].reset_index(drop=True)
    X[inst] = sub[feat_cols].values
    Y[inst] = sub["target_regime"].values

NUM_TAGS = int(Y.max()) + 1

# ── Train/test split ───────────────────────────────────────────────────────────
X_train = torch.tensor(X[:, :TRAIN_LEN, :])
Y_train = torch.tensor(Y[:, :TRAIN_LEN])
X_test  = torch.tensor(X[:, TRAIN_LEN:, :])
Y_test  = Y[:, TRAIN_LEN:]     # numpy for plotting
LEN_TEST = X_test.shape[1]

# ── DataLoader ────────────────────────────────────────────────────────────────
class SeqTagDataset(Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y
    def __len__(self):
        return self.X.size(0)
    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_loader = DataLoader(SeqTagDataset(X_train, Y_train),
                          batch_size=BATCH_SIZE, shuffle=True)

# ── Model definition ──────────────────────────────────────────────────────────
class UniLSTMTagger(nn.Module):
    def __init__(self, feat_dim, hidden_dim, num_layers, num_tags, dropout):
        super().__init__()
        self.lstm = nn.LSTM(feat_dim, hidden_dim,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=False,
                            dropout=dropout)
        self.fc = nn.Linear(hidden_dim, num_tags)
    def forward(self, x):
        out, _ = self.lstm(x)   # (B, T, H)
        return self.fc(out)     # (B, T, num_tags)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UniLSTMTagger(len(feat_cols), HIDDEN_SIZE, NUM_LAYERS, NUM_TAGS, DROPOUT).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# ── Training ─────────────────────────────────────────────────────────────────
for epoch in range(1, NUM_EPOCHS+1):
    model.train()
    total_loss = 0.0
    for feats, tags in train_loader:
        feats, tags = feats.to(device), tags.to(device)
        logits = model(feats)
        loss = criterion(logits.view(-1, NUM_TAGS), tags.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch:02d} — Loss: {total_loss/len(train_loader):.4f}")

# ── Inference & shift back ────────────────────────────────────────────────────
model.eval()
with torch.no_grad():
    logits_test = model(X_test.to(device))                # (n_inst, LEN_TEST, C)
    preds_test  = logits_test.argmax(dim=2).cpu().numpy()

# drop last HORIZON preds to align with real-time
preds_shift = preds_test[:, :-HORIZON]

# ── Plotting ──────────────────────────────────────────────────────────────────
def get_segments(reg):
    changes = np.flatnonzero(reg[1:] != reg[:-1])
    starts  = np.concatenate(([0], changes+1))
    ends    = np.concatenate((changes, [len(reg)-1]))
    return list(zip(starts, ends, reg[starts]))

true_cmap = ListedColormap(["#ff0000","#00ff00"])  # 0→bear, 2→bull
pred_cmap = ListedColormap(["#cc0000","#00cc00"])

for inst in range(n_inst):
    true_seq = Y_test[inst, HORIZON:]        # aligned true regimes at time t+H
    pred_seq = preds_shift[inst]
    price    = price_df.iloc[100+TRAIN_LEN+HORIZON : 
                              100+TRAIN_LEN+HORIZON+len(pred_seq), inst].values

    acc = (pred_seq == true_seq).mean()
    print(f"Inst {inst:02d} Test acc (H={HORIZON}): {acc:.3f}")

    fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,6), sharex=True)
    # TRUE
    for s,e,lbl in get_segments(true_seq):
        ax1.axvspan(s, e, color=true_cmap(lbl//2), alpha=0.4, linewidth=0)
    ax1.plot(price, 'k-', label='Price'); ax1.set_title(f"Inst {inst:02d} — TRUE regimes")
    # PREDICTED
    for s,e,lbl in get_segments(pred_seq):
        ax2.axvspan(s, e, color=pred_cmap(lbl//2), alpha=0.4, linewidth=0)
    ax2.plot(price, 'k-', label='Price'); ax2.set_title(f"Inst {inst:02d} — PREDICTED (H={HORIZON})")
    plt.tight_layout()
    plt.show()
